
# Bucket-Brigade QRAM Read (toy 2-qubit address)

This notebook builds a tiny QRAM read circuit in Qiskit using a bucket-brigade inspired control structure. The address register has two qubits and the data register has one qubit, giving four classical memory cells with contents `d = [0, 1, 1, 0]`. The goal is to realize the map

$$\sum_i lpha_i |i
angle_A |0
angle_D \longmapsto \sum_i lpha_i |i
angle_A |d_i
angle_D,$$

without treating QRAM as a black-box oracle. Instead, we explicitly condition on the address bits to flip the data qubit wherever the stored classical bit is 1.



## Phase 1 – Problem instance and bucket-brigade logic

- Address qubits: `a0` (MSB), `a1` (LSB)
- Data qubit: `d`
- Classical memory: `d = [0, 1, 1, 0]` so `00 → 0`, `01 → 1`, `10 → 1`, `11 → 0`
- Ideal QRAM map: \(\sum_i lpha_i |i
angle_A |0
angle_D 	o \sum_i lpha_i |i
angle_A |d_i
angle_D\).

Bucket-brigade hardware would route the address down a binary tree of switches, storing the path. For two address qubits the tree has a root controlled by `a0` and two leaves controlled by `a1`. Here we compress that routing into multi-controlled X gates: for each address whose classical content is 1 we apply an `mcx` to the data qubit, using temporary X gates to emulate controls on `|0
angle` where needed.


In [ ]:

from __future__ import annotations
from typing import Iterable, List

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector

num_addr = 2
num_data = 1
memory_bits = [0, 1, 1, 0]

# Qubit layout: [a0, a1, d]
address_qubits = list(range(num_addr))
data_qubit = num_addr

def prepare_address(qc: QuantumCircuit, addr_bits: str) -> None:
    '''Load a classical address (MSB-first string) into the address register.'''
    if len(addr_bits) != num_addr:
        raise ValueError(f"Expected {num_addr} bits, got {len(addr_bits)}")
    for qb, bit in zip(address_qubits, addr_bits):
        if bit == "1":
            qc.x(qb)

def apply_qram_read(qc: QuantumCircuit, d: Iterable[int]) -> None:
    '''Apply bucket-brigade-inspired QRAM read based on classical memory d.'''
    for i, bit in enumerate(d):
        if bit != 1:
            continue
        addr_bits = format(i, f"0{num_addr}b")  # MSB -> LSB string

        zero_controls: List[int] = [qb for qb, addr_bit in zip(address_qubits, addr_bits) if addr_bit == "0"]
        for qb in zero_controls:
            qc.x(qb)

        qc.mcx(address_qubits, data_qubit)

        for qb in zero_controls:
            qc.x(qb)



## Phase 2 – Basis checks

The circuit should flip the data qubit only for addresses `01` and `10`. We test each basis address using the statevector simulator and report the most likely bitstring (displayed as `a0 a1 d`).


In [ ]:

from collections import Counter


def simulate_basis_address(addr_bits: str):
    qc = QuantumCircuit(num_addr + num_data)
    prepare_address(qc, addr_bits)
    apply_qram_read(qc, memory_bits)
    state = Statevector.from_instruction(qc)
    probs = state.probabilities_dict()
    # Reorder bitstrings from qiskit's |q2 q1 q0> to |a0 a1 d>
    reordered = {bits[::-1]: p for bits, p in probs.items() if p > 1e-9}
    return Counter(reordered)

for addr in ["00", "01", "10", "11"]:
    counts = simulate_basis_address(addr)
    top_state = counts.most_common(1)[0]
    print(f"Address |{addr}> -> dominant state |{top_state[0]}> with probability {top_state[1]:.3f}")



## Phase 3 – Superposition address

Prepare a uniform superposition over all addresses, apply the same QRAM read, and inspect the resulting entangled state. The expected output for `d = [0, 1, 1, 0]` is

\[
	frac{1}{2}ig(|00
angle|0
angle + |01
angle|1
angle + |10
angle|1
angle + |11
angle|0
angleig).
\]


In [ ]:

superposition = QuantumCircuit(num_addr + num_data)
for qb in address_qubits:
    superposition.h(qb)

apply_qram_read(superposition, memory_bits)

state = Statevector.from_instruction(superposition)
probs = state.probabilities_dict()
filtered = {bits[::-1]: p for bits, p in probs.items() if p > 1e-9}

print("Non-zero basis states (shown as a0 a1 d):")
for bits, prob in sorted(filtered.items()):
    print(f"|{bits}>: {prob:.3f}")



## Phase 4 – Resource estimates

We can get a sense of the cost of this logical QRAM read by transpiling the three-qubit circuit and inspecting depth and operation counts. The numbers below use Qiskit's default transpiler settings and keep the two `mcx` gates explicit.


In [ ]:

transpiled = transpile(superposition)
print("Circuit depth:", transpiled.depth())
print("Operation counts:", transpiled.count_ops())
